In [ ]:
import xarray
import openeo

import openeo
from openeo.processes import exp, array_element, log 
from openeo.extra.spectral_indices.spectral_indices import append_index
from openeo.udf.debug import inspect
from datetime import datetime
from dateutil.relativedelta import *

In [ ]:
connection = openeo.connect(
    url="openeo-dev.vito.be"
    # url="openeo.vito.be"
).authenticate_oidc()

# S2 Collection

In [ ]:
start_date      = '2022-01-03'
end_date        = '2022-01-08'
spatial_extent  = {'west': -74.06810760, 'east': -73.90597343, 'south': 4.689864510, 'north': 4.724080996, 'crs': 'epsg:4326'}

# Selecting B03 was needed to make sure that the angles bands have 10 m resolution
s2_cube = connection.load_collection(
    'SENTINEL2_L2A_SENTINELHUB',
    spatial_extent = spatial_extent,
    temporal_extent = [start_date, end_date],
    bands = ['B03', 'sunAzimuthAngles', 'sunZenithAngles'] 
)

# DEM data (COPERNICUS_30)

In [ ]:
dem_cube = connection.load_collection(
    "COPERNICUS_30",
    spatial_extent = spatial_extent,
    temporal_extent=["2010-01-01", "2030-12-31"],
)

dem_cube = dem_cube.max_time()
dem_cube = dem_cube.resample_cube_spatial(s2_cube)
merged_cube = s2_cube.merge_cubes(dem_cube)

# Hillshade process with UDF

In [ ]:
udf_code = """

from openeo.udf import XarrayDataCube
from openeo.udf.debug import inspect
import numpy as np
from hillshade.hillshade import hillshade


def rasterize(azimuth, resolution=None):
    # Convert the azimuth into its components on the XY-plane. Depending on the value of the
    # azimuth either the x or the y component of the resulting vector is scaled to 1, so that
    # it can be used conveniently to walk a grid.

    azimuth = np.deg2rad(azimuth)
    xdir, ydir = np.sin(azimuth), np.cos(azimuth)

    if resolution is not None:
        xdir = xdir * resolution[0]
        ydir = ydir * resolution[1]

    slope = ydir / xdir
    if slope < 1. and slope > -1.:
        xdir = 1.
        ydir = slope
    else:
        xdir = 1. / slope
        ydir = 1.
    return xdir, ydir


def _run_shader(sun_zenith, sun_azimuth, elevation_model, resolution_x, resolution_y):

    azimuth = np.nanmean(sun_azimuth.astype(np.float32))
    zenith = np.nanmean(sun_zenith.astype(np.float32))
    if np.isnan(azimuth):
        shadow = np.zeros(elevation_model.shape) + 255
    else:
        resolution = (float(resolution_x), float(resolution_y))
        ray_xdir, ray_ydir = rasterize(azimuth, resolution)

        # Assume chunking is already done by Dask
        ystart = 0
        yend = elevation_model.shape[0]

        # Make sure inputs have the right data type
        zenith = float(zenith)
        ray = (float(ray_xdir), float(ray_ydir))
        shadow = hillshade(elevation_model.astype(np.float32),
                           resolution,
                           zenith,
                           ray,
                           ystart,
                           yend)
        shadow = shadow.reshape(elevation_model.shape)
        shadow[np.isnan(sun_azimuth)] = 255
    return shadow


def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    in_xarray = cube.get_array()
    sun_zenith = in_xarray.sel({"bands": "sunZenithAngles"}).values.astype(np.float32)
    sun_azimuth = in_xarray.sel({"bands": "sunAzimuthAngles"}).values.astype(np.float32)
    elevation_model = in_xarray.sel({"bands": "DEM"}).values.astype(np.float32)
    res_y = in_xarray.coords["y"][int(len(in_xarray.coords["y"])/2)+1] - in_xarray.coords["y"][int(len(in_xarray.coords["y"])/2)]
    res_x = in_xarray.coords["x"][int(len(in_xarray.coords["x"])/2)+1] - in_xarray.coords["x"][int(len(in_xarray.coords["x"])/2)]

    shadow = _run_shader(sun_zenith, sun_azimuth, elevation_model, res_x, res_y)
    cube.get_array().values[0] = shadow
    return cube

"""

In [ ]:
process = openeo.UDF(code=udf_code, runtime="Python", data={"from_parameter": "x"})

# Shadow mask replaces the first band (B03) in the merged cube
hillshaded = merged_cube.apply_neighborhood(process=process,
                                            size=[{"dimension":"t","value": "P1D"},
                                                  {"dimension": "x", "unit": "px", "value": 256},
                                                  {"dimension": "y", "unit": "px", "value": 256}],
                                            overlap=[{"dimension": "x", "unit": "px", "value": "8"},
                                                     {"dimension": "y", "unit": "px", "value": "8"}])


my_job  = hillshaded.send_job(title="hillshaded")
results = my_job.start_and_wait().get_results()
results.download_files("hilshaded")

hillshaded = hillshaded.rename_labels("bands", ["hillshade_mask", "sunAzimuthAngles", "sunZenithAngles", "DEM"])


In [ ]:
my_job.logs()